In [2]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from modules.ContrastiveLosses import clr_loss, anomclr_loss, anomclr_plus_loss #Please download the modules from https://github.com/bmdillon/AnomalyCLR
# Also download the EventLevelAnomalyAugmentations.py and import the module to utlize the augmentations

In [9]:
sig = np.load("../signalAto4l.npy")
sig[:,:,0] /= np.mean(sig[:,:,0].flatten()[(0 < sig[:,:,0].flatten())])
sig[:,:,1] /= np.max(sig[:,:,1])
sig[:,:,2] /= np.max(sig[:,:,2])
sig = np.swapaxes(sig, 1, 2)

In [5]:
sig.shape

(4000000, 7, 19)

In [8]:
bkg = np.load("../sm_taged.npy")
bkg[:,:,0] /= np.mean(bkg[:,:,0].flatten()[(0 < bkg[:,:,0].flatten())])
bkg[:,:,1] /= np.max(bkg[:,:,1])
bkg[:,:,2] /= np.max(bkg[:,:,2])
bkg = np.swapaxes(bkg, 1, 2)

In [10]:
sig.shape

(10000, 3, 19)

In [4]:
n_epochs = 100
batch_size = 128
trnsize = 10000
valsize = 2000

In [5]:
trndat = bkg[0:trnsize,:,:]
valdat = sig[0:valsize,:,:]

In [6]:
trndat.shape

(10000, 7, 19)

In [7]:
######
# Definding Transformer Encoder and NN
######

class Transformer( nn.Module ):
    
    # define and intialize the structure of the neural network
    def __init__( self, input_dim, model_dim, output_dim, n_heads, dim_feedforward, n_layers, learning_rate, n_head_layers=2, head_norm=False, dropout=0.1, opt="adam" ):
        super().__init__()
        # define hyperparameters
        self.input_dim = input_dim
        self.model_dim = model_dim
        self.output_dim = output_dim
        self.n_heads = n_heads
        self.dim_feedforward = dim_feedforward
        self.n_layers = n_layers
        self.learning_rate = learning_rate
        self.n_head_layers = n_head_layers
        self.head_norm = head_norm
        self.dropout = dropout
        # define subnetworks
        self.embedding = nn.Linear(input_dim, model_dim)
        self.transformer = nn.TransformerEncoder(nn.TransformerEncoderLayer(model_dim, n_heads, dim_feedforward=dim_feedforward, dropout=dropout), n_layers)
        # head_layers have output_dim
        if n_head_layers == 0:
            self.head_layers = []
        else:
            if head_norm: self.norm_layers = nn.ModuleList([nn.LayerNorm(model_dim)])
            self.head_layers = nn.ModuleList([nn.Linear(model_dim, output_dim)])
            for i in range(n_head_layers-1):
                if head_norm: self.norm_layers.append(nn.LayerNorm(output_dim))
                self.head_layers.append(nn.Linear(output_dim, output_dim))
        # option to use adam or sgd
        if opt == "adam":
            self.optimizer = torch.optim.Adam( self.parameters(), lr=self.learning_rate )
        if opt == "sgdca" or opt == "sgdslr" or opt == "sgd":
            self.optimizer = torch.optim.SGD( self.parameters(), lr=self.learning_rate, momentum=0.9 )

    def forward(self, inpt, mask=None, use_mask=False, use_continuous_mask=False, mult_reps=False):
        '''
        input here is (batch_size, n_constit, 7)
        but transformer expects (n_constit, batch_size, 7) so we need to transpose
        if use_mask is True, will mask out all inputs with pT=0
        '''
        assert not (use_mask and use_continuous_mask)
        # make a copy
        x = inpt + 0.   
        # (batch_size, n_constit)
        if use_mask: pT_zero = x[:,:,0] == 0 
        # (batch_size, n_constit)
        if use_continuous_mask: pT = x[:,:,0] 
        if use_mask:
            mask = self.make_mask(pT_zero).to(x.device)
        elif use_continuous_mask:
            mask = self.make_continuous_mask(pT).to(x.device)
        else:
            mask = None
        x = torch.transpose(x, 0, 1)
        # (n_constit, batch_size, model_dim)
        x = self.embedding(x)               
        x = self.transformer(x, mask=mask)
        if use_mask:
            # set masked constituents to zero
            # otherwise the sum will change if the constituents with 0 pT change
            x[torch.transpose(pT_zero, 0, 1)] = 0
        elif use_continuous_mask:
            # scale x by pT, so that function is IR safe
            # transpose first to get correct shape
            x *= torch.transpose(pT, 0, 1)[:,:,None]
        # sum over sequence dim
        # (batch_size, model_dim)
        x = x.sum(0)                        
        return self.head(x, mult_reps)


    def head(self, x, mult_reps):
        '''
        calculates output of the head if it exists, i.e. if n_head_layer>0
        returns multiple representation layers if asked for by mult_reps = True
        input:  x shape=(batchsize, model_dim)
                mult_reps boolean
        output: reps shape=(batchsize, output_dim)                  for mult_reps=False
                reps shape=(batchsize, number_of_reps, output_dim)  for mult_reps=True
        '''
        relu = nn.ReLU()
            # return representations from multiple layers for evaluation
        if mult_reps == True:   
            if self.n_head_layers > 0:
                reps = torch.empty(x.shape[0], self.n_head_layers+1, self.output_dim)
                reps[:, 0] = x
                for i, layer in enumerate(self.head_layers):
                    # only apply layer norm on head if chosen
                    if self.head_norm: x = self.norm_layers[i](x)       
                    x = relu(x)
                    x = layer(x)
                    reps[:, i+1] = x
                # shape (n_head_layers, output_dim)
                return reps  
            # no head exists -> just return x in a list with dimension 1
            else:  
                reps = x[:, None, :]
                # shape (batchsize, 1, model_dim)
                return reps  
        # return only last representation for contrastive loss
        else:  
            for i, layer in enumerate(self.head_layers):  # will do nothing if n_head_layers is 0
                if self.head_norm: x = self.norm_layers[i](x)
                x = relu(x)
                x = layer(x)
            # shape either (model_dim) if no head, or (output_dim) if head exists
            return x  


    def forward_batchwise( self, x, batch_size, use_mask=False, use_continuous_mask=False):
        device = next(self.parameters()).device
        with torch.no_grad():
            if self.n_head_layers == 0:
                rep_dim = self.model_dim
                number_of_reps = 1
            elif self.n_head_layers > 0:
                rep_dim = self.output_dim
                number_of_reps = self.n_head_layers+1
            out = torch.empty( x.size(0), number_of_reps, rep_dim )
            idx_list = torch.split( torch.arange( x.size(0) ), batch_size )
            for idx in idx_list:
                output = self(x[idx].to(device), use_mask=use_mask, use_continuous_mask=use_continuous_mask, mult_reps=True).detach().cpu()
                out[idx] = output
        return out


    def make_mask(self, pT_zero):
        '''
        Input: batch of bools of whether pT=0, shape (batchsize, n_constit)
        Output: mask for transformer model which masks out constituents with pT=0, shape (batchsize*n_transformer_heads, n_constit, n_constit)
        mask is added to attention output before softmax: 0 means value is unchanged, -inf means it will be masked
        '''
        n_constit = pT_zero.size(1)
        pT_zero = torch.repeat_interleave(pT_zero, self.n_heads, axis=0)
        pT_zero = torch.repeat_interleave(pT_zero[:,None], n_constit, axis=1)
        mask = torch.zeros(pT_zero.size(0), n_constit, n_constit)
        mask[pT_zero] = -np.inf
        return mask
    
    
    def make_continuous_mask(self, pT):
        '''
        Input: batch of pT values, shape (batchsize, n_constit)
        Output: mask for transformer model: -1/pT, shape (batchsize*n_transformer_heads, n_constit, n_constit)
        mask is added to attention output before softmax: 0 means value is unchanged, -inf means it will be masked
        intermediate values mean it is partly masked
        This function implements IR safety in the transformer
        '''
        n_constit = pT.size(1)
        pT_reshape = torch.repeat_interleave(pT, self.n_heads, axis=0)
        pT_reshape = torch.repeat_interleave(pT_reshape[:,None], n_constit, axis=1)
        #mask = -1/pT_reshape
        mask = 0.5*torch.log( pT_reshape )
        return mask


In [8]:
torch.set_num_threads(2)

# set gpu device
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )

mask = True
cmask = False

model_dim = 136
input_dim = 7
output_dim = 136
n_heads = 4
dim_feedforward = 136
n_layers = 4
learning_rate = 0.00005
n_head_layers = 2
opt = 'adam'
temperature = 0.1

net = Transformer( input_dim, model_dim, output_dim, n_heads, dim_feedforward, 
                    n_layers, learning_rate, n_head_layers, dropout=0.1, opt=opt )



# set gpu device
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )

# send network 
net.to( device )

Transformer(
  (embedding): Linear(in_features=7, out_features=136, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=136, out_features=136, bias=True)
        )
        (linear1): Linear(in_features=136, out_features=136, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=136, out_features=136, bias=True)
        (norm1): LayerNorm((136,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((136,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=136, out_features=136, bias=True)
        )
        (linear1): Linear(in_features=136

In [ ]:
# the loop

trclrlosses = []
vlclrlosses = []


for epoch in range( n_epochs ):
    
    lossestr = []
    lossesvl = []

    # re-batch the data on each epoch
    indices_list = torch.split( torch.randperm( trndat.shape[0] ), batch_size )
    vl_indices_list = torch.split( torch.randperm( valdat.shape[0] ), batch_size)
    
    for i, indices in enumerate( indices_list ):
        net.optimizer.zero_grad()
        x_i = trndat[indices,:,:]
        x_j = x_i.copy()
        
        x_i = torch.Tensor( x_i ).transpose(1,2).to( device )
        x_j = torch.Tensor( x_j ).transpose(1,2).to( device )
        
        z_i = net( x_i, use_mask=mask, use_continuous_mask=cmask )
        z_j = net( x_j, use_mask=mask, use_continuous_mask=cmask )
        
        
        loss = clr_loss( z_i, z_j, temperature ).to( device )
        loss.backward()
        net.optimizer.step()
        lossestr.append( loss.detach().cpu().numpy() )
        
    loss_tr = np.mean( np.array( lossestr ) )
    trclrlosses.append( loss_tr )
    

    
    for i, indices in enumerate(vl_indices_list):
        net.eval()  
        with torch.no_grad(): 
            
            v_i = valdat[indices,:,:]
            v_j = v_i.copy()
        
            v_i = torch.Tensor( v_i ).transpose(1,2).to( device )
            v_j = torch.Tensor( v_j ).transpose(1,2).to( device )
        
            y_i = net( v_i, use_mask=mask, use_continuous_mask=cmask )
            y_j = net( v_j, use_mask=mask, use_continuous_mask=cmask )
            
            lossvl = clr_loss( y_i, y_j, temperature ).to( device )
            
            lossesvl.append( lossvl.detach().cpu().numpy() )  
    
    vl_loss_e = np.mean( np.array( lossesvl ) )
    vlclrlosses.append( vl_loss_e )

    net.train()  
    
    if epoch%50==0:
        torch.save(net.state_dict(), "clr_model" + str(epoch) + ".pt")
        np.save( "clr_tr_losses.npy", trclrlosses )